<a href="https://colab.research.google.com/github/rafs-santos/Learn_Methods/blob/main/Torch_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a basic MLP with pytorch

Classify Iris dataset 

In [166]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn import datasets

import numpy as np

### Load data in the custom class to use structure of pytorch (dataset, dataloader)

In [167]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.labels = torch.from_numpy(labels)
        self.data = torch.from_numpy(data)
        print(self.data.dtype)
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx,:], self.labels[idx]

### Split data_train and data_test and use dataloader

In [168]:
from sklearn.model_selection import StratifiedShuffleSplit

iris = datasets.load_iris()
X = np.float32(iris.data)
y = np.float32(iris.target).reshape((len(iris.target),1))


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
for train_index, test_index in sss.split(X, y):
  X_train, X_test = X[train_index,:], X[test_index,:]
  y_train, y_test = y[train_index], y[test_index]

#print(type(X_train[0,0]))
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=len(y_train), shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=len(y_test), shuffle=True)

torch.float32
torch.float32


In [169]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(dim_in, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_out)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
dim_in, dim_hidden, dim_out = 4, 15, 1
model = NeuralNetwork(dim_in, dim_hidden, dim_out).to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=1, bias=True)
  )
)


In [170]:
# Mean Square Error
lossfunc = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)

In [171]:
def train(model, dataloader, lossfunc, optmizer):
  model.train()

  cumloss = 0.0
  for X, y in dataloader:
    X = X.to(device)
    y = y.to(device)

    pred = model(X)
    loss = lossfunc(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cumloss += loss.item()

  return cumloss/len(dataloader)

def test(model, dataloader, lossfunc):
  model.eval()

  cumloss = 0.0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)

      pred = model(X)
      loss = lossfunc(pred, y)

      print(np.round(pred.detach().cpu().numpy().T))
      print(y.detach().cpu().numpy().T)
      cumloss += loss.item()

  return cumloss/len(dataloader)

In [172]:
epochs = 1000

for t in range(epochs):
  train_loss = train(model, train_dataloader, lossfunc, optimizer)
  if t % 500 == 0:
    print(f"Epoch: {t}; Train Loss: {train_loss}")

test_loss = test(model, test_dataloader, lossfunc)
print(f"Test Loss: {test_loss}")

Epoch: 0; Train Loss: 6.356400012969971
Epoch: 500; Train Loss: 0.4714565575122833
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[2. 2. 2. 1. 0. 0. 1. 2. 2. 1. 0. 2. 0. 1. 0. 1. 1. 0. 0. 1. 2. 1. 0.]]
Test Loss: 0.3845616579055786
